## Análise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a (1) extração de características, (2) seleção de características e (3) classificação dos dados

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com a diferença de 0.2 Hz uma da outra): 8.0, 8.2, ..., 15.6, 15.8;
- Taxa de amostragem: 250 Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP

1. Criar o objeto `MNE` a partir dos dados dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 - 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);
    a) 0.0 - 0.5 segundos e 2.5 - 3.0 segundos ocorre apenas ruído;
    b) 0.5 - 2.5 segundos ocorre sinal SSVEP (com ruídos)
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações.
    - Os dados devem ser plotados no domínio da frequência (após a transformada de Fourier). O FFT pode ser realizado pela biblioteca `scipy.fft`.
    - Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `BETA` é o *signal-to-noise ratio* (SNR).
São dois tipos de características SNR que podem ser implementadas: SNR de banda estreita (`narrow-SNR`) e SNR de banda larga (`wide-band SNR`).

Uma boa prática, é considerar o ruído das medidas de `SNR`, uma vez que os dados `SSVEP` não estão estimulados durante os períodos de 0 a 0,5 segundos e de 2,5 a 3 segundos. O ruído pode afetar a precisão das medidas de `SNR` e, portanto, é aconselhável levar isso em consideração.

Vamos realizar todos esses cálculos com dados fictícios:

In [179]:
import numpy as np

ch_ideal = ["Pz", "PO3", "PO5", "PO4", "PO6", "POz", "O1", "Oz", "O2"] #canais ideais
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]


data = np.load("./datasets/beta/data2.npy")


labels = np.load("./datasets/beta/labels2.npy")

data.shape,labels.shape

((160, 64, 750), (160,))

In [180]:
import mne

n_channels = 64
sfreq = 250
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=(['eeg'] * len(ch_names)))

ev = {str(value): index  for index, value in enumerate(sorted(set(labels)))}

labels.max(), labels.min()

(15.8, 8.0)

Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`. 

In [181]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(160, dtype=int),
    le.fit_transform(labels))
)


mne_data = mne.EpochsArray(data, info, events, event_id=ev)
filtered_mne_data = mne.EpochsArray(data, info, events, event_id=ev).filter(7.9, 16)


Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 7.9 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.90
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.90 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 10240 out of 10240 | elapsed:    1.4s finished


In [182]:
# # Estimando o ruído de fundo

# # intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
# base_start = 0
# base_end = 125
# rest_start = 625
# rest_end = 750

# # armazena uma lista com as médias de potência para cada canal
# noise_power = []
# # consideramos a primeira amostra (1º target, 1º trial)
# for channel_data in data[0, :, :]:
#     fft_result = np.fft.fft(channel_data)
#     # densidade espectral de potência (PSD)
#     psd = np.abs(fft_result) ** 2
#     # média da potência nos intervalos de tempo sem estímulo
#     base_power = np.mean(psd[base_start:base_end])
#     rest_power = np.mean(psd[rest_start:rest_end])
#     # média das duas médias de potência obtidas anteriormente
#     mean_noise_power = (base_power + rest_power) / 2
#     noise_power.append(mean_noise_power)
# #média das médias de potência de todos os canais para estimar o ruído de fundo
# estimated_background_noise = np.mean(noise_power)
# estimated_background_noise

sr = 250

target_frequencies = np.arange(8, 16, 0.2)

target_amplitudes = []

for channel_data in mne_data.get_data():
    target_per_channel = [] # Armazena as amplitudes para cada frequência alvo
    
    for sign in channel_data:
        fft_result = np.fft.fft(sign) # Transformada de Fourier
        psd = np.abs(fft_result) ** 2 # Densidade espectral de potência (PSD)
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr) # Frequências
         
        target_amplitudes_trial = [] 
        
        for target_frequency in target_frequencies:
            index = np.argmin(np.abs(frequencies - target_frequency)) # Índice da frequência alvo
            amplitude = np.sqrt(psd[index]) # Amplitude da frequência alvo
            target_amplitudes_trial.append(amplitude) # Armazena a amplitude da frequência alvo
        
        target_per_channel.append(target_amplitudes_trial) # Armazena as amplitudes para cada frequência alvo
    
    target_amplitudes.append(target_per_channel) # Armazena as amplitudes para cada canal



target_amplitudes = np.array(target_amplitudes) # Converte para array numpy



sn = filtered_mne_data.copy().crop(tmin=0.0, tmax=0.5)# 
mean = filtered_mne_data.copy().crop(tmin=0.5, tmax=2.5)
end = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)

noise_power = []

fft_sn =  np.abs(np.fft.fft(sn)) ** 2 # Densidade espectral de potência (PSD)
fft_end = np.abs(np.fft.fft(end)) ** 2 



# média da potência nos intervalos de tempo sem estímulo
base_power = np.mean(fft_sn, axis=-1)
rest_power = np.mean(fft_end, axis=-1)

# média das duas médias de potência obtidas anteriorment
snr = np.mean([base_power + rest_power])

snr


C:\Users\caio-\AppData\Local\Temp\ipykernel_24476\83043227.py:59: RuntimeWarning: tmax is not in time interval. tmax is set to <class 'mne.epochs.EpochsArray'>.tmax (2.996 s)
  end = filtered_mne_data.copy().crop(tmin=2.5, tmax=3.0)


1855.2387318680223

Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [177]:
target_amplitudes_adjusted = np.abs(target_amplitudes - snr) #ajuste da amplitude do sinal
 
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / snr) # razao sinal ruido de banda estreita
print(narrow_band_SNR) 
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted) # potencia total
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
print(wide_band_SNR)
print(wide_band_SNR.shape)

data = np.array([narrow_band_SNR, wide_band_SNR]) # array com as duas razoes sinal ruido
print(data.shape)
data = data.swapaxes(0, 1) # troca de eixos
print(data.shape)
data = data.swapaxes(1, 2) # troca de eixos
print(data.shape)
data = data.reshape(data.shape[0], data.shape[1], data.shape[2] * data.shape[3]) # reshape

#salva os dados
np.save("./datasets/beta/data2_filtered.npy", data)

[[[-0.00380946 -0.00479036 -0.00479036 ... -0.00181056 -0.00204601
   -0.00204601]
  [-0.00380241 -0.0046575  -0.0046575  ... -0.00185963 -0.00209516
   -0.00209516]
  [-0.00370597 -0.00443876 -0.00443876 ... -0.00178978 -0.00189245
   -0.00189245]
  ...
  [-0.00408539 -0.00701864 -0.00701864 ... -0.00181484 -0.00263654
   -0.00263654]
  [-0.00447996 -0.00530451 -0.00530451 ... -0.00149292 -0.00375224
   -0.00375224]
  [-0.00473444 -0.00506979 -0.00506979 ... -0.00153413 -0.00389193
   -0.00389193]]

 [[-0.00135948 -0.00237819 -0.00237819 ... -0.00289992 -0.00239066
   -0.00239066]
  [-0.00097136 -0.00234852 -0.00234852 ... -0.00320174 -0.00220419
   -0.00220419]
  [-0.00077009 -0.00224808 -0.00224808 ... -0.00327011 -0.00233739
   -0.00233739]
  ...
  [-0.0009429  -0.00450935 -0.00450935 ... -0.00138677 -0.00760409
   -0.00760409]
  [-0.00093998 -0.00506148 -0.00506148 ... -0.00111696 -0.00569485
   -0.00569485]
  [-0.00071605 -0.00509259 -0.00509259 ... -0.00119855 -0.00606193
   -0.

Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (brigatória);
- `wide-band SNR` (brigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores
`160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `160, 192`.

### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as caracteríscas que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicado por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.